In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, matthews_corrcoef, roc_auc_score, roc_curve, confusion_matrix
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score, fbeta_score
from sklearn.metrics import brier_score_loss, average_precision_score, precision_recall_curve, make_scorer
from sklearn.calibration import  calibration_curve
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, cross_validate, StratifiedKFold
from sklearn.pipeline import Pipeline

from sklearn.experimental import enable_iterative_imputer

from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.feature_selection import RFE

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

from scipy.stats import randint, uniform

In [2]:
def tw_accuracy(clf, X_test, y_test):
  y_pred =  clf.predict_proba(X_test)[:,1]
  check = (y_pred >= 0.75) | (y_pred <= 0.25)
  y_pred = clf.predict(X_test)
  y_pred = y_pred[check]
  if len(y_pred) == 0 or len(y_test[check]) == 0:
    return 1
  return accuracy_score(y_test[check], y_pred)

def tw_sens(clf, X_test, y_test):
  y_pred =  clf.predict_proba(X_test)[:,1]
  check = (y_pred >= 0.75) | (y_pred <= 0.25)
  y_pred = clf.predict(X_test)
  y_pred = y_pred[check]
  if len(y_pred) == 0 or len(y_test[check]) == 0:
    return 1
  return recall_score(y_test[check], y_pred)

def tw_spec(clf, X_test, y_test):
  y_pred =  clf.predict_proba(X_test)[:,1]
  check = (y_pred >= 0.75) | (y_pred <= 0.25)
  y_pred = clf.predict(X_test)
  y_pred = y_pred[check]
  if len(y_pred) == 0 or len(y_test[check]) == 0:
    return 1
  return recall_score(y_test[check], y_pred, pos_label=0)

def tw_ppv(clf, X_test, y_test):
  y_pred =  clf.predict_proba(X_test)[:,1]
  check = (y_pred >= 0.75) | (y_pred <= 0.25)
  y_pred = clf.predict(X_test)
  y_pred = y_pred[check]
  if len(y_pred) == 0 or len(y_test[check]) == 0:
    return 1
  return precision_score(y_test[check], y_pred)

def tw_npv(clf, X_test, y_test):
  y_pred =  clf.predict_proba(X_test)[:,1]
  check = (y_pred >= 0.75) | (y_pred <= 0.25)
  y_pred = clf.predict(X_test)
  y_pred = y_pred[check]
  if len(y_pred) == 0 or len(y_test[check]) == 0:
    return 1
  return precision_score(y_test[check], y_pred, pos_label=0)

def coverage(clf, X_test, y_test):
  y_pred =  clf.predict_proba(X_test)[:,1]
  check = (y_pred >= 0.75) | (y_pred <= 0.25)
  y_pred = y_pred[check]
  return len(y_pred)/len(y_test)

def nb(y_true, y_proba, th=0.5):
  prop = len(y_true[y_true == 1])/len(y_true)
  y_pred = (y_proba >= th).astype(int)
  sens = recall_score(y_true, y_pred)
  spec = recall_score(y_true, y_pred, pos_label=0)
  return (sens*prop - (1-spec)*(1-prop)*th/(1-th))/prop

In [3]:
def std_brier(y_test, y_proba):
  val = np.mean(y_proba**4)
  val -= 4*np.mean(y_proba**3 * y_test)
  val += 6*np.mean(y_proba**2 * y_test)
  val -= 4*np.mean(y_proba * y_test)
  val += np.mean(y_test) 
  val -= brier_score_loss(y_test, y_proba)**2
  return val

def std_auc(auc, num_pos, num_neg):
  prop = num_pos/(num_pos + num_neg)
  n = num_pos + num_neg
  se = np.sqrt(auc*(1-auc)*(1 + (n/2 - 1)*(1 - auc)/(2- auc) + (n/2 -1)*auc/(1+auc))/(n**2*prop*(1-prop)))
  return se**2

def std_nb(sens, spec, prop, n, th=0.25):
  w = (1-prop)/prop*th/(1-th)
  return 1/n * ( sens*(1-sens)/prop + w**2 * spec*(1-spec)/(1-prop) + w**2 * (1-spec)**2/(prop*(1-prop)))

def std_score(score, sample_size):
    return np.sqrt(score*(1-score)/sample_size)

In [4]:
from sklearn.base import BaseEstimator, ClassifierMixin

class ThresholdClassifier(BaseEstimator, ClassifierMixin):
    
    def __init__(self, threshold, feature):
        self.threshold = threshold
        self.feature = feature
        
    def fit(self, X, y):
        self.X = X
        self.y = y
        self.classes_ = np.unique(self.y)
        return self
        
    def predict(self, X):
        y_pred = X[self.feature] >= self.threshold
        y_pred = y_pred.astype(int).values
        return y_pred.reshape((y_pred.shape[0],1))
    
    def fit_predict(self, X, y):
        self.fit(X, y)
        return self.predict(X)
    
    def predict_proba(self, X):
        probs = np.zeros((len(X),2))
        y_pred = self.predict(X)
        probs[:,y_pred] = 1
        return probs

In [5]:
class RuleClassifier(BaseEstimator, ClassifierMixin):
    
    def __init__(self, rules):
        self.rules = rules
        
    def fit(self, X, y):
        self.X = X
        self.y = y
        self.classes_ = np.unique(self.y)
        return self
        
    def predict(self, X):
        y_pred = []
        
        for idx in range(X.shape[0]):
            check_all = True
            for rule in self.rules:
                check = True
                
                for i in range(len(rule)-1):
                    if rule[i,1] == ">=":
                        check = check and (X.iloc[idx,:][rule[i,0]] >= float(rule[i,2]))
                    else:
                        check = check and (X.iloc[idx,:][rule[i,0]] <= float(rule[i,2]))

                if check:
                    y_pred.append(1 if rule[-1,2] >= rule[-1,0] else 0)
                    check_all = False
                    break

            if check_all:
                y_pred.append(0)
        return np.array(y_pred)
    
    def fit_predict(self, X, y):
        self.fit(X, y)
        return self.predict(X)
    
    def predict_proba(self, X):
        probs = np.zeros((len(X),2))
        y = []
        for idx in range(X.shape[0]):
            check_all = True
            for rule in self.rules:
                check = True
                for i in range(len(rule)-1):
                    if rule[i,1] == ">=":
                        check &= (X.iloc[idx,:][rule[i,0]] >= float(rule[i,2]))
                    else:
                        check &= (X.iloc[idx,:][rule[i,0]] <= float(rule[i,2]))

                if check:
                    probs[idx,0] = rule[-1,0]
                    probs[idx,1] = rule[-1,2]
                    check_all = False
                    break
             
            if check_all:
                probs[idx,:] = [0.5,0.5]
        return probs

In [6]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer, MaxAbsScaler, RobustScaler, Normalizer

from sklearn.base import BaseEstimator, TransformerMixin

class FlexibleScaler(BaseEstimator, TransformerMixin):
    def __init__(self, scaler=None):
        self.scaler = scaler
        self.check = False
        
            
    def __assign_scaler(self):
        if self.scaler == 'min-max':
            self.method = MinMaxScaler()
        elif self.scaler == 'standard':
            self.method = StandardScaler()
        elif self.scaler == 'yeo-johnson':
            self.method = PowerTransformer(method='yeo-johnson')
        elif self.scaler == 'box-cox':
            self.method = PowerTransformer(method='box-cox')
        elif self.scaler == 'max-abs':
            self.method = MaxAbsScaler()
        elif self.scaler == 'robust':
            self.method = RobustScaler()
        elif self.scaler == 'normalize':
            self.method = Normalizer()
        else:
            self.method = None
        self.check = True
    
    def fit_transform(self, X, y=None, **fit_params):
        if not self.check:
            self.__assign_scaler()
        if self.method is None:
            return X
        return self.method.fit_transform(X, y, **fit_params)
        
    def fit(self, X):
        if not self.check:
            self.__assign_scaler()
        if self.method is None:
            return X
        self.method.fit(X)
        
    def transform(self, X):
        if not self.check:
            self.__assign_scaler()
        if self.method is None:
            return X
        return self.method.transform(X)

In [7]:
dati = pd.read_excel("dati_sepsi_complete.xlsx", header=0, sheet_name="Foglio1")
features = ["diagn_finale", "age", "WBC", "NEU", "MONO", "MDW", "hsPCR", "sex", "LI#", "EO#", "BA#",
            "RBC", "HGB", "HCT", "MCV", "MCH", "MCHC", "RDW", "PLT"]
dati = dati.loc[:, features]
dati.columns = ["DIAGNOSI", "Age", "WBC", "Neutrophils", "Monocytes", "MDW",
                "CRP", "Sex", "Lymphocytes", "Eosinophils", "Basophils", "RBC", "HGB",
               "HCT", "MCV", "MCH", "MCHC", "RDW", "PLT"]
dati = dati[dati["Lymphocytes"] > 0]

dati["NLR"] = dati["Neutrophils"]/dati["Lymphocytes"]
dati = dati.dropna()

dati.loc[:,"DIAGNOSI"] = dati["DIAGNOSI"].apply(lambda x: 1 if ("SE" in x or "se" in x) else 0 )
dati["DIAGNOSI"].value_counts()/dati.shape[0]
dati = dati.apply(pd.to_numeric, errors='coerce')
dati = dati.dropna()

dati = dati[dati >= 0]

X_pa = dati.drop(columns="DIAGNOSI")
y_pa = dati.loc[:,"DIAGNOSI"]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_pa, y_pa, random_state=0, stratify=y_pa)

In [9]:
baselines = ["MDW", "SepsisIndex", "COMPOSER", "PDR","CRP"]
sens = [0.90, 0.90, 0.8, 0.81,0.67]
twsens = [0,0,0,0.94,0]
spec = [0.89, 0.93, 0.93, 0.98,0.94]
twspec = [0,0,0,0.98,0]
ppv = [0.28, 0.45, 0.30, 0.65,0.37]
twppv = [0,0,0,0.65,0]
npv = [1, 1, 0.98, 0.99,0.98]
twnpv = [0,0,0,1,0]
covs = [0,0,0,0.90,0]

for i, _ in enumerate(baselines):
    print("Sens (%s): %.2f +- %2f" % (baselines[i], sens[i], 1.96*std_score(sens[i],X_test.shape[0])))
    print("Spec (%s): %.2f +- %2f" % (baselines[i], spec[i], 1.96*std_score(spec[i],X_test.shape[0])))
    print("PPV (%s): %.2f +- %2f" % (baselines[i], ppv[i], 1.96*std_score(ppv[i],X_test.shape[0])))
    print("NPV (%s): %.2f +- %2f" % (baselines[i], npv[i], 1.96*std_score(npv[i],X_test.shape[0])))
    print("HC Sens (%s): %.2f +- %2f" % (baselines[i], twsens[i], 1.96*std_score(twsens[i],X_test.shape[0])))
    print("HC Spec (%s): %.2f +- %2f" % (baselines[i], twspec[i], 1.96*std_score(twspec[i],X_test.shape[0])))
    print("HC PPV (%s): %.2f +- %2f" % (baselines[i], twppv[i], 1.96*std_score(twppv[i],X_test.shape[0])))
    print("HC NPV (%s): %.2f +- %2f" % (baselines[i], twnpv[i], 1.96*std_score(twnpv[i],X_test.shape[0])))
    print("Coverage (%s): %.2f +- %2f" % (baselines[i], covs[i], 1.96*std_score(covs[i],X_test.shape[0])))
    print()

Sens (MDW): 0.90 +- 0.027627
Spec (MDW): 0.89 +- 0.028814
PPV (MDW): 0.28 +- 0.041348
NPV (MDW): 1.00 +- 0.000000
HC Sens (MDW): 0.00 +- 0.000000
HC Spec (MDW): 0.00 +- 0.000000
HC PPV (MDW): 0.00 +- 0.000000
HC NPV (MDW): 0.00 +- 0.000000
Coverage (MDW): 0.00 +- 0.000000

Sens (SepsisIndex): 0.90 +- 0.027627
Spec (SepsisIndex): 0.93 +- 0.023496
PPV (SepsisIndex): 0.45 +- 0.045814
NPV (SepsisIndex): 1.00 +- 0.000000
HC Sens (SepsisIndex): 0.00 +- 0.000000
HC Spec (SepsisIndex): 0.00 +- 0.000000
HC PPV (SepsisIndex): 0.00 +- 0.000000
HC NPV (SepsisIndex): 0.00 +- 0.000000
Coverage (SepsisIndex): 0.00 +- 0.000000

Sens (COMPOSER): 0.80 +- 0.036836
Spec (COMPOSER): 0.93 +- 0.023496
PPV (COMPOSER): 0.30 +- 0.042200
NPV (COMPOSER): 0.98 +- 0.012892
HC Sens (COMPOSER): 0.00 +- 0.000000
HC Spec (COMPOSER): 0.00 +- 0.000000
HC PPV (COMPOSER): 0.00 +- 0.000000
HC NPV (COMPOSER): 0.00 +- 0.000000
Coverage (COMPOSER): 0.00 +- 0.000000

Sens (PDR): 0.81 +- 0.036127
Spec (PDR): 0.98 +- 0.012892
PPV

In [10]:
mindray = SVC(**{"C" : 1.291549665014884,  "coef0" : 0.0,  "degree" : 2,  "kernel" :  "rbf" ,  "probability" : True})
mindray.fit(X_train, y_train)
y_proba_mindray = mindray.predict_proba(X_test)
y_pred_mindray = mindray.predict(X_test)

print("Sens: %.2f" % recall_score(y_test, y_pred_mindray))
print("Spec: %.2f" % recall_score(y_test, y_pred_mindray, pos_label=0))
print("PPV: %.2f" % precision_score(y_test, y_pred_mindray))
print("NPV: %.2f" % precision_score(y_test, y_pred_mindray, pos_label=0))
print("AUC: %.2f" % roc_auc_score(y_test, y_proba_mindray[:,1]))

Sens: 0.19
Spec: 0.99
PPV: 0.50
NPV: 0.96
AUC: 0.91


In [11]:
from joblib import dump, load


scoring = {
    'sensitivity': make_scorer(recall_score),
    'specificity': make_scorer(recall_score, pos_label=0),
    'ppv': make_scorer(precision_score),
    'npv': make_scorer(precision_score, pos_label=0),
    'auc': make_scorer(roc_auc_score, needs_proba=True),
    'brier': make_scorer(brier_score_loss, needs_proba=True),
    'a-pr': make_scorer(average_precision_score, needs_proba=True),
    'twsens': tw_sens,
    'twspec': tw_spec,
    'twppv': tw_ppv,
    'twnpv': tw_npv,
    'coverage': coverage,
    'nb': make_scorer(nb, th=0.25, needs_proba=True)
}

    

In [12]:
from joblib import dump, load

best_models = {}

names = ['LR','SVM','RF','DT','XGB']

for name in names:
    best_models[name] = load("./" + name + ".joblib")
    print(best_models[name])
best_models

Pipeline(steps=[('scl', FlexibleScaler(scaler='max-abs')),
                ('sel',
                 RFE(estimator=LogisticRegression(max_iter=1000),
                     n_features_to_select=17)),
                ('clf',
                 LogisticRegression(C=2.40549802303397, class_weight='balanced',
                                    l1_ratio=0.44919773821837206,
                                    max_iter=10000, penalty='l1',
                                    random_state=0, solver='saga'))])
Pipeline(steps=[('scl', FlexibleScaler(scaler='robust')),
                ('sel',
                 RFE(estimator=LogisticRegression(max_iter=1000),
                     n_features_to_select=16)),
                ('clf',
                 SVC(C=0.37416998033422555, class_weight='balanced', degree=2,
                     kernel='poly', max_iter=1000, probability=True,
                     random_state=0))])
Pipeline(steps=[('scl', FlexibleScaler(scaler='standard')),
                ('sel',
    

{'LR': Pipeline(steps=[('scl', FlexibleScaler(scaler='max-abs')),
                 ('sel',
                  RFE(estimator=LogisticRegression(max_iter=1000),
                      n_features_to_select=17)),
                 ('clf',
                  LogisticRegression(C=2.40549802303397, class_weight='balanced',
                                     l1_ratio=0.44919773821837206,
                                     max_iter=10000, penalty='l1',
                                     random_state=0, solver='saga'))]),
 'SVM': Pipeline(steps=[('scl', FlexibleScaler(scaler='robust')),
                 ('sel',
                  RFE(estimator=LogisticRegression(max_iter=1000),
                      n_features_to_select=16)),
                 ('clf',
                  SVC(C=0.37416998033422555, class_weight='balanced', degree=2,
                      kernel='poly', max_iter=1000, probability=True,
                      random_state=0))]),
 'RF': Pipeline(steps=[('scl', FlexibleScaler(scaler='st

In [13]:
results = pd.DataFrame(np.zeros((len(scoring.keys()),len(names))), columns=names, index=scoring.keys())
results_str = pd.DataFrame(np.empty((len(scoring.keys()),len(names))), columns=names, index=scoring.keys(), dtype=str)
best_models["MindraySVM"] = mindray
names = ['LR','SVM','RF','DT','XGB',"MindraySVM"]
for i, name in enumerate(names):
    print(name)
    best_models[name].fit(X_train, y_train)
    for score in scoring:
        results.loc[score, name] = scoring[score](best_models[name], X_test, y_test)
        y_pred = best_models[name].predict(X_test)
        y_proba = best_models[name].predict_proba(X_test)[:,1]
        if score in ["sensitivity","twsens","specificity","twspec","ppv","twppv","npv","twnpv","coverage"]:
            ci = 1.96*std_score(results.loc[score,name],X_test.shape[0])
        elif score == "auc" or "a-pr":
            ci = 1.96*std_auc(results.loc[score,name],X_test[y_test==1].shape[0],X_test[y_test==0].shape[0])
        elif score == "brier":
            ci = 1.96*std_brier(y_test,y_proba)
        elif score == "nb":
            ci = 1.96*std_nb(results.loc["sensitivity",name], results.loc["specificity",name], X_test[y_test==1].shape[0], X_test.shape[0], th=0.25)
            
        
        print(score, ": %.2f +- %.2f" % (results.loc[score, name],ci))
        results_str.loc[score, name] = "%.2f (%.2f)" % (results.loc[score,name], ci)
    print()
    
#results.to_csv('results.csv', float_format='%.2f')
results_str

LR
sensitivity : 0.90 +- 0.03
specificity : 0.94 +- 0.02
ppv : 0.43 +- 0.05
npv : 1.00 +- 0.01
auc : 0.96 +- 0.00
brier : 0.05 +- 0.00
a-pr : 0.78 +- 0.01
twsens : 0.90 +- 0.03
twspec : 0.96 +- 0.02
twppv : 0.51 +- 0.05
twnpv : 0.99 +- 0.01
coverage : 0.94 +- 0.02
nb : 0.22 +- 0.01

SVM
sensitivity : 0.86 +- 0.03
specificity : 0.98 +- 0.01
ppv : 0.64 +- 0.04
npv : 0.99 +- 0.01
auc : 0.95 +- 0.00
brier : 0.02 +- 0.00
a-pr : 0.81 +- 0.00
twsens : 0.81 +- 0.04
twspec : 0.98 +- 0.01
twppv : 0.62 +- 0.04
twnpv : 0.99 +- 0.01
coverage : 0.98 +- 0.01
nb : 0.62 +- 0.01

RF
sensitivity : 0.81 +- 0.04
specificity : 0.97 +- 0.02
ppv : 0.57 +- 0.05
npv : 0.99 +- 0.01
auc : 0.97 +- 0.00
brier : 0.03 +- 0.00
a-pr : 0.77 +- 0.01
twsens : 0.86 +- 0.03
twspec : 0.99 +- 0.01
twppv : 0.71 +- 0.04
twnpv : 0.99 +- 0.01
coverage : 0.92 +- 0.03
nb : 0.35 +- 0.01

DT
sensitivity : 0.86 +- 0.03
specificity : 0.91 +- 0.03
ppv : 0.32 +- 0.04
npv : 0.99 +- 0.01
auc : 0.91 +- 0.00
brier : 0.05 +- 0.00
a-pr : 0.67 

,LR,SVM,RF,DT,XGB,MindraySVM
sensitivity,0.90 (0.03),0.86 (0.03),0.81 (0.04),0.86 (0.03),0.86 (0.03),0.19 (0.04)
specificity,0.94 (0.02),0.98 (0.01),0.97 (0.02),0.91 (0.03),0.97 (0.02),0.99 (0.01)
ppv,0.43 (0.05),0.64 (0.04),0.57 (0.05),0.32 (0.04),0.60 (0.05),0.50 (0.05)
npv,1.00 (0.01),0.99 (0.01),0.99 (0.01),0.99 (0.01),0.99 (0.01),0.96 (0.02)
auc,0.96 (0.00),0.95 (0.00),0.97 (0.00),0.91 (0.00),0.98 (0.00),0.91 (0.00)
brier,0.05 (0.00),0.02 (0.00),0.03 (0.00),0.05 (0.00),0.03 (0.00),0.04 (0.00)
a-pr,0.78 (0.01),0.81 (0.00),0.77 (0.01),0.67 (0.01),0.83 (0.00),0.44 (0.01)
twsens,0.90 (0.03),0.81 (0.04),0.86 (0.03),0.85 (0.03),0.90 (0.03),0.21 (0.04)
twspec,0.96 (0.02),0.98 (0.01),0.99 (0.01),0.99 (0.01),0.98 (0.01),0.99 (0.01)
twppv,0.51 (0.05),0.62 (0.04),0.71 (0.04),0.77 (0.04),0.72 (0.04),0.50 (0.05)


In [14]:
results_str.to_csv("res_int_str.csv")

In [15]:
for i, name in enumerate(names):
    print(name)
    best_models[name].fit(X_pa, y_pa)
    dump(best_models[name], name+"_fitted.joblib")

LR
SVM
RF
DT
XGB
MindraySVM


# External

In [16]:
best_models = {}

names = ['LR','SVM','RF','DT','XGB']

for name in names:
    best_models[name] = load("./" + name + "_fitted.joblib")
    print(best_models[name])
best_models

Pipeline(steps=[('scl', FlexibleScaler(scaler='max-abs')),
                ('sel',
                 RFE(estimator=LogisticRegression(max_iter=1000),
                     n_features_to_select=17)),
                ('clf',
                 LogisticRegression(C=2.40549802303397, class_weight='balanced',
                                    l1_ratio=0.44919773821837206,
                                    max_iter=10000, penalty='l1',
                                    random_state=0, solver='saga'))])
Pipeline(steps=[('scl', FlexibleScaler(scaler='robust')),
                ('sel',
                 RFE(estimator=LogisticRegression(max_iter=1000),
                     n_features_to_select=16)),
                ('clf',
                 SVC(C=0.37416998033422555, class_weight='balanced', degree=2,
                     kernel='poly', max_iter=1000, probability=True,
                     random_state=0))])
Pipeline(steps=[('scl', FlexibleScaler(scaler='standard')),
                ('sel',
    

{'LR': Pipeline(steps=[('scl', FlexibleScaler(scaler='max-abs')),
                 ('sel',
                  RFE(estimator=LogisticRegression(max_iter=1000),
                      n_features_to_select=17)),
                 ('clf',
                  LogisticRegression(C=2.40549802303397, class_weight='balanced',
                                     l1_ratio=0.44919773821837206,
                                     max_iter=10000, penalty='l1',
                                     random_state=0, solver='saga'))]),
 'SVM': Pipeline(steps=[('scl', FlexibleScaler(scaler='robust')),
                 ('sel',
                  RFE(estimator=LogisticRegression(max_iter=1000),
                      n_features_to_select=16)),
                 ('clf',
                  SVC(C=0.37416998033422555, class_weight='balanced', degree=2,
                      kernel='poly', max_iter=1000, probability=True,
                      random_state=0))]),
 'RF': Pipeline(steps=[('scl', FlexibleScaler(scaler='st

In [17]:
cols = ["Age", "WBC", "Neutrophils", "Monocytes", "MDW",
            "CRP", "Sex", "Lymphocytes", "Eosinophils", "Basophils", "RBC", "HGB",
            "HCT", "MCV", "MCH", "MCHC", "RDW", "PLT", "DIAGNOSI"]

Xs = {}
ys = {}

In [18]:
text = "PD"
data = pd.read_excel("DB 3 _ Database MDW PD.xlsx")
feats = ["Patient birthday", "WBC", "NE#", "MO#", "MDW", "CRP", "Sex", "LI#", "EO#", "BA#",
         "RBC", "HGB", "HCT", "MCV", "MCH", "MCHC", "RDW", "PLT", "Sepsis-3"]
data = data.loc[:, feats].dropna()

data["Patient birthday"] = np.round((np.round((pd.to_datetime('now') - pd.to_datetime(data['Patient birthday'])) / np.timedelta64(1, 'D')))/365)
data.columns = cols

data.loc[:,"DIAGNOSI"] = data["DIAGNOSI"].apply(lambda x: 0 if ("No" in x) else 1 )

data.loc[:, "Sex"] = data.loc[:,"Sex"].apply(lambda x: 0 if x == "Maschio" else 1 if x == "Femmina" else np.nan)
data = data.dropna().apply(pd.to_numeric, errors='coerce').dropna()

data = data[data["Lymphocytes"] > 0]
data["NLR"] = data["Neutrophils"]/data["Lymphocytes"]
    
y = data.loc[:,"DIAGNOSI"]
X = data.drop(columns="DIAGNOSI")

Xs["PD-ICU"] = X
ys["PD-ICU"] = y

In [19]:
text = "PA"
data = pd.read_excel("DB 2 _ Database Palermo MDW-ICU - 21.05.2020 final.xlsx")

feats = ["AGE", "WBC (10^3/ul) Beckmann", "NEUTROFILI (#)", "MONOCITI (#)", "MDW", "PCR (mg/L) Beckmann", "SESSO",
             "    LINFOCITI (#)", "    EOSINOFILI (#)", "    BASOFILI (#)", "    ERITROCITI (10^6/ul)",
             "    EMOGLOBINA (g/dl)", "    EMATOCRITO (%)",
             "    MCV (fL)", "    MCH (pg)",
             "    MCHC (g/dL)", "    RDW (fL)", " PIASTRINE (10^3/ul)", "CATEGORIA RICOVERO"]

data = data.loc[:,feats].dropna()

data.columns = cols

data.loc[:,"DIAGNOSI"] = data["DIAGNOSI"].apply(lambda x: 1 if ("SE" in x or "se" in x) else 0 )
print(data["DIAGNOSI"].value_counts()/data.shape[0])

data.loc[:, "Sex"] = data.loc[:,"Sex"].apply(lambda x: 0 if x == "M" else 1 if x == "F" else np.nan)
data = data.dropna().apply(pd.to_numeric, errors='coerce').dropna()

data = data[data["Lymphocytes"] > 0]
data["NLR"] = data["Neutrophils"]/data["Lymphocytes"]
    
y = data.loc[:,"DIAGNOSI"]
X = data.drop(columns="DIAGNOSI")

Xs["PA-ICU"] = X
ys["PA-ICU"] = y

DIAGNOSI
0    0.677083
1    0.322917
Name: count, dtype: float64


In [20]:
text="UD"

data = pd.read_excel("df_Udine.xlsx")
feats = ["Età", "WBC", "NE#", "MO#", "MDW",
            "PCR", "Sesso", "LI#", "EO#", "BA#", "RBC", "HGB",
            "HCT", "MCV", "MCH", "MCHC", "RDW", "PLT", "sepsi (sì=1)"]

data["sepsi (sì=1)"] = data["sepsi (sì=1)"].apply(lambda x: 1 if x == 1 else 0)

data = data.loc[:,feats].dropna()

data.columns = cols
data.loc[:, "Sex"] = data.loc[:,"Sex"].apply(lambda x: 0 if x == "Maschio" else 1 if x == "Femmina" else np.nan)
data = data.dropna().apply(pd.to_numeric, errors='coerce').dropna()
data["MDW"] = data["MDW"] + 1.5

data = data[data["Lymphocytes"] > 0]
data["NLR"] = data["Neutrophils"]/data["Lymphocytes"]

y = data.loc[:,"DIAGNOSI"]
X = data.drop(columns="DIAGNOSI")

Xs["UD-ED"] = X
ys["UD-ED"] = y

In [21]:
text = "AR"

data = pd.read_excel("df_Arezzo.xlsx", decimal=',')
feats = ["Età", "WBC", "NE#", "MO#", "MDW",
            "PCR", "Sesso", "LI#", "EO#", "BA#", "RBC", "HGB",
            "HCT", "MCV", "MCH", "MCHC", "RDW", "PLT", "Sepsi (1/0)"]

data = data.loc[:,feats].dropna()

data.columns = cols
data.loc[:, "Sex"] = data.loc[:,"Sex"].apply(lambda x: 0 if x == "M" else 1 if x == "F" else np.nan)
data = data.dropna().apply(pd.to_numeric, errors='coerce').dropna()

data["MDW"] = data["MDW"] + 1.5

data = data[data["Lymphocytes"] > 0]
data["NLR"] = data["Neutrophils"]/data["Lymphocytes"]
    
y = data.loc[:,"DIAGNOSI"]
X = data.drop(columns="DIAGNOSI")

Xs["AR-ED"] = X
ys["AR-ED"] = y

In [22]:
text = "OGSA"

data = pd.read_csv("OGSA_imputed.csv", decimal=',')
feats = ["Age", "WBC", "Neutrophils", "Monocytes", "MDW (imputed)",
            "CRP", "Sex", "Lymphocytes", "Eosinophils", "Basophils", "RBC", "HGB",
            "HCT", "MCV", "MCH", "MCHC", "RDW", "PLT", "NLR", "DIAGNOSI"]
data = data.loc[:,feats]

cols = ["Age", "WBC", "Neutrophils", "Monocytes", "MDW",
            "CRP", "Sex", "Lymphocytes", "Eosinophils", "Basophils", "RBC", "HGB",
            "HCT", "MCV", "MCH", "MCHC", "RDW", "PLT", "NLR", "DIAGNOSI"]
data.columns = cols
    
y = data.loc[:,"DIAGNOSI"]
X = data.drop(columns="DIAGNOSI")

Xs["OGSA-ICU"] = X
ys["OGSA-ICU"] = y

In [23]:
names = ['LR','SVM','RF','DT','XGB']
datasets = ["AR-ED","OGSA-ICU","PA-ICU","PD-ICU","UD-ED"]
index = pd.MultiIndex.from_product([names,datasets], names=["Model","Dataset"])
columns = list(scoring.keys())

results = pd.DataFrame(np.zeros((len(index),len(columns))), columns=columns, index=index)
results_str = pd.DataFrame(np.empty((len(index),len(columns))), columns=columns, index=index, dtype=str)

In [24]:
clf = RuleClassifier([np.array([["MDW", ">=", 25.5], [0.06, 0, 0.94]]),
                              np.array([["MDW", "<=", 24], ["Neutrophils", "<=", 8.6], [0.98, 0, 0.02]]),
                              np.array([["MDW", "<=", 20.5], ["Neutrophils", ">=", 8.6], [1, 0, 0]])])

In [25]:
for k in Xs.keys():
    print(k)
    for i, name in enumerate(names):
        print(name)
        for score in scoring:
            results.loc[(name,k),score] = scoring[score](best_models[name], Xs[k], ys[k])
            y_pred = best_models[name].predict(Xs[k])
            y_proba = best_models[name].predict_proba(Xs[k])[:,1]
            if score in ["sensitivity","twsens","specificity","twspec","ppv","twppv","npv","twnpv","coverage"]:
                ci = 1.96*std_score(results.loc[(name,k),score],Xs[k].shape[0])
            elif score == "auc" or "a-pr":
                ci = 1.96*std_auc(results.loc[(name,k),score],Xs[k][ys[k]==1].shape[0],Xs[k][ys[k]==0].shape[0])
            elif score == "brier":
                ci = 1.96*std_brier(ys[k],y_proba)
            elif score == "nb":
                ci = 1.96*std_nb(results.loc[(name,k),"sensitivity"], results.loc[(name,k),"specificity"], Xs[k][ys[k]==1].shape[0], Xs[k].shape[0], th=0.25)
                
            
            print(score, ": %.2f +- %.2f" % (results.loc[(name,k),score],ci))
            results_str.loc[(name,k),score] = "%.2f (%.2f)" % (results.loc[(name,k),score], ci)
        print()

PD-ICU
LR
sensitivity : 0.72 +- 0.02
specificity : 0.65 +- 0.02
ppv : 0.51 +- 0.02
npv : 0.82 +- 0.02
auc : 0.76 +- 0.00
brier : 0.25 +- 0.00
a-pr : 0.63 +- 0.00
twsens : 0.77 +- 0.02
twspec : 0.68 +- 0.02
twppv : 0.57 +- 0.02
twnpv : 0.84 +- 0.02
coverage : 0.77 +- 0.02
nb : 0.49 +- 0.00

SVM
sensitivity : 0.62 +- 0.02
specificity : 0.74 +- 0.02
ppv : 0.55 +- 0.02
npv : 0.80 +- 0.02
auc : 0.72 +- 0.00
brier : 0.22 +- 0.00
a-pr : 0.62 +- 0.00
twsens : 0.57 +- 0.02
twspec : 0.80 +- 0.02
twppv : 0.57 +- 0.02
twnpv : 0.80 +- 0.02
coverage : 0.91 +- 0.01
nb : 0.38 +- 0.00

RF
sensitivity : 0.70 +- 0.02
specificity : 0.73 +- 0.02
ppv : 0.56 +- 0.02
npv : 0.83 +- 0.02
auc : 0.76 +- 0.00
brier : 0.20 +- 0.00
a-pr : 0.60 +- 0.00
twsens : 0.76 +- 0.02
twspec : 0.80 +- 0.02
twppv : 0.67 +- 0.02
twnpv : 0.87 +- 0.01
coverage : 0.55 +- 0.02
nb : 0.50 +- 0.00

DT
sensitivity : 0.65 +- 0.02
specificity : 0.74 +- 0.02
ppv : 0.56 +- 0.02
npv : 0.81 +- 0.02
auc : 0.72 +- 0.00
brier : 0.25 +- 0.00
a-pr 

In [26]:
results_str.to_csv("res_ext.csv")

In [28]:
results_str

sensitivity  specificity          ppv          npv  \
Model Dataset                                                        
LR    AR-ED     0.82 (0.04)  0.61 (0.05)  0.41 (0.05)  0.91 (0.03)   
      OGSA-ICU  1.00 (0.00)  0.60 (0.10)  0.14 (0.07)  1.00 (0.00)   
      PA-ICU    0.82 (0.09)  0.66 (0.11)  0.50 (0.11)  0.90 (0.07)   
      PD-ICU    0.72 (0.02)  0.65 (0.02)  0.51 (0.02)  0.82 (0.02)   
      UD-ED     0.65 (0.03)  0.85 (0.02)  0.24 (0.03)  0.97 (0.01)   
SVM   AR-ED     0.66 (0.05)  0.73 (0.05)  0.45 (0.06)  0.87 (0.04)   
      OGSA-ICU  0.83 (0.07)  0.70 (0.09)  0.15 (0.07)  0.99 (0.02)   
      PA-ICU    0.73 (0.10)  0.55 (0.11)  0.40 (0.11)  0.83 (0.09)   
      PD-ICU    0.62 (0.02)  0.74 (0.02)  0.55 (0.02)  0.80 (0.02)   
      UD-ED     0.59 (0.03)  0.91 (0.02)  0.31 (0.03)  0.97 (0.01)   
RF    AR-ED     0.55 (0.06)  0.78 (0.05)  0.45 (0.06)  0.84 (0.04)   
      OGSA-ICU  0.83 (0.07)  0.69 (0.09)  0.15 (0.07)  0.98 (0.02)   
      PA-ICU    0.91 (0.07)  0.79 (0.09)  0.65 (0.11)  0.95 (0.05)   
      PD-ICU    0.70 (0.02)  0.73 (0.02)  0.56 (0.02)  0.83 (0.02)   
      UD-ED     0.60 (0.03)  0.90 (0.02)  0.29 (0.03)  0.97 (0.01)   
DT    AR-ED     0.80 (0.04)  0.55 (0.06)  0.37 (0.05)  0.90 (0.03)   
      OGSA-ICU  0.83 (0.07)  0.68 (0.09)  0.14 (0.07)  0.98 (0.02)   
      PA-ICU    0.82 (0.09)  0.79 (0.09)  0.62 (0.11)  0.91 (0.06)   
      PD-ICU    0.65 (0.02)  0.74 (0.02)  0.56 (0.02)  0.81 (0.02)   
      UD-ED     0.65 (0.03)  0.85 (0.02)  0.24 (0.03)  0.97 (0.01)   
XGB   AR-ED     0.61 (0.05)  0.78 (0.05)  0.48 (0.06)  0.86 (0.04)   
      OGSA-ICU  0.83 (0.07)  0.71 (0.09)  0.16 (0.07)  0.99 (0.02)   
      PA-ICU    0.82 (0.09)  0.85 (0.08)  0.69 (0.10)  0.92 (0.06)   
      PD-ICU    0.65 (0.02)  0.79 (0.02)  0.61 (0.02)  0.82 (0.02)   
      UD-ED     0.52 (0.03)  0.91 (0.02)  0.30 (0.03)  0.96 (0.01)   

                        auc        brier         a-pr       twsens  \
Model Dataset                                                        
LR    AR-ED     0.78 (0.00)  0.28 (0.00)  0.54 (0.00)  0.85 (0.04)   
      OGSA-ICU  0.83 (0.02)  0.36 (0.03)  0.23 (0.02)  1.00 (0.00)   
      PA-ICU    0.89 (0.00)  0.20 (0.01)  0.85 (0.00)  0.95 (0.05)   
      PD-ICU    0.76 (0.00)  0.25 (0.00)  0.63 (0.00)  0.77 (0.02)   
      UD-ED     0.87 (0.00)  0.13 (0.00)  0.34 (0.00)  0.74 (0.03)   
SVM   AR-ED     0.75 (0.00)  0.19 (0.00)  0.52 (0.00)  0.61 (0.05)   
      OGSA-ICU  0.79 (0.02)  0.14 (0.01)  0.33 (0.03)  0.83 (0.07)   
      PA-ICU    0.75 (0.01)  0.26 (0.01)  0.72 (0.01)  0.70 (0.10)   
      PD-ICU    0.72 (0.00)  0.22 (0.00)  0.62 (0.00)  0.57 (0.02)   
      UD-ED     0.81 (0.00)  0.07 (0.00)  0.35 (0.00)  0.54 (0.03)   
RF    AR-ED     0.75 (0.00)  0.17 (0.00)  0.43 (0.00)  0.08 (0.03)   
      OGSA-ICU  0.86 (0.01)  0.21 (0.02)  0.28 (0.02)  1.00 (0.00)   
      PA-ICU    0.95 (0.00)  0.11 (0.00)  0.89 (0.00)  1.00 (0.00)   
      PD-ICU    0.76 (0.00)  0.20 (0.00)  0.60 (0.00)  0.76 (0.02)   
      UD-ED     0.86 (0.00)  0.09 (0.00)  0.34 (0.00)  0.62 (0.03)   
DT    AR-ED     0.73 (0.00)  0.33 (0.00)  0.41 (0.00)  0.87 (0.04)   
      OGSA-ICU  0.79 (0.02)  0.28 (0.02)  0.18 (0.02)  0.83 (0.07)   
      PA-ICU    0.83 (0.01)  0.21 (0.01)  0.69 (0.01)  0.86 (0.08)   
      PD-ICU    0.72 (0.00)  0.25 (0.00)  0.56 (0.00)  0.73 (0.02)   
      UD-ED     0.80 (0.00)  0.13 (0.00)  0.26 (0.00)  0.70 (0.03)   
XGB   AR-ED     0.77 (0.00)  0.21 (0.00)  0.52 (0.00)  0.65 (0.05)   
      OGSA-ICU  0.82 (0.02)  0.25 (0.02)  0.32 (0.02)  0.83 (0.07)   
      PA-ICU    0.93 (0.00)  0.13 (0.00)  0.87 (0.00)  0.86 (0.08)   
      PD-ICU    0.78 (0.00)  0.22 (0.00)  0.65 (0.00)  0.66 (0.02)   
      UD-ED     0.83 (0.00)  0.10 (0.00)  0.39 (0.00)  0.55 (0.03)   

                     twspec        twppv        twnpv     coverage  \
Model Dataset                                                        
LR    AR-ED     0.62 (0.05)  0.45 (0.06)  0.92 (0.03)  0.80 (0.04)   
      OGSA-ICU  0.50 (0.

In [42]:
vars(best_models["XGB"]["clf"])

{'use_label_encoder': True,
 'n_estimators': 909,
 'objective': 'binary:logistic',
 'max_depth': 24,
 'learning_rate': None,
 'verbosity': None,
 'booster': None,
 'tree_method': None,
 'gamma': 3.322238526617727,
 'min_child_weight': None,
 'max_delta_step': None,
 'subsample': 0.5505881150542372,
 'colsample_bytree': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'scale_pos_weight': 42.007536979061065,
 'base_score': None,
 'missing': nan,
 'num_parallel_tree': None,
 'random_state': 0,
 'n_jobs': None,
 'monotone_constraints': None,
 'interaction_constraints': None,
 'importance_type': None,
 'gpu_id': None,
 'validate_parameters': None,
 'predictor': None,
 'enable_categorical': False,
 'kwargs': {'eval_metric': 'logloss',
  'alpha': 2.0569836187772728,
  'eta': 0.03799451709864034,
  'lambda': 4.912874799125968},
 'classes_': array([0, 1]),
 'n_classes_': 2,
 '_le': XGBoostLabelEncoder(),
 '_Booster': <xgboost.core.Booster at